# Quickstart

In [16]:
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

load_dotenv()

True

In [21]:
class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


parser = PydanticOutputParser(pydantic_object=Classification)
tagging_prompt = ChatPromptTemplate.from_template(
    """
    Extract the desired information from the following passage.

    Wrap the output in `json` tags\n{format_instructions}

    Passage:
    {input}
    """
).partial(format_instructions=parser.get_format_instructions())

llm = ChatOpenAI(model="qwen-plus")

In [22]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain = tagging_prompt | llm | parser
response = chain.invoke(inp)
response

Classification(sentiment='positive', aggressiveness=1, language='es')

# Finer control

In [23]:
class Classification(BaseModel):
    sentiment: str = Field(enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5])
    language: str = Field(enum=["spanish", "english", "french", "german", "italian"])

In [24]:
parser = PydanticOutputParser(pydantic_object=Classification)
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

{format_instructions}

Passage:
{input}
""").partial(format_instructions=parser.get_format_instructions())

llm = ChatOpenAI(model="qwen-max")

In [25]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain = tagging_prompt | llm | parser
response = chain.invoke(inp)
response

Classification(sentiment='happy', aggressiveness=1, language='spanish')

In [26]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
chain = tagging_prompt | llm | parser
response = chain.invoke(inp)
response

Classification(sentiment='sad', aggressiveness=5, language='spanish')

In [27]:
inp = "Weather is ok here, I can go outside without much more than a coat"
chain = tagging_prompt | llm | parser
response = chain.invoke(inp)
response

Classification(sentiment='neutral', aggressiveness=1, language='english')